In [1]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

In [2]:
TRAIN_PATH = "data/processed_data/train_df.csv"
TEST_PATH = "data/processed_data/test_df.csv"
SUB_DIR = "data/submission"

# Loading data

In [3]:
def load_nyc_taxi_fare(path, col_types, chunksize=None, 
                       datetime_format="%Y-%m-%d %H:%M:%S UTC",
                       utc=False,
                       convert_to_timezone=None):
    chunk_iter = pd.read_csv(path, usecols=col_types.keys(), dtype=col_types, chunksize=chunksize)
    
    if chunksize is None:
        chunk_iter["pickup_datetime"] = pd.to_datetime(chunk_iter["pickup_datetime"], 
                                                       utc=utc, format=datetime_format)
        # convert to different timezone
        if convert_to_timezone is not None:
            chunk_iter["pickup_datetime"] = chunk_iter["pickup_datetime"].dt.tz_convert(convert_to_timezone)
        return chunk_iter
    
    df_list = []
    # use tqdm to monitor progress
    # It would take extremely long time if format were not used.
    for df_chunk in tqdm(chunk_iter):
        df_chunk["pickup_datetime"] = pd.to_datetime(df_chunk["pickup_datetime"], 
                                                     utc=utc, format=datetime_format)
        # convert to different timezone
        if convert_to_timezone is not None:
            df_chunk["pickup_datetime"] = df_chunk["pickup_datetime"].dt.tz_convert(convert_to_timezone)
        
        df_list.append(df_chunk)
        
    return pd.concat(df_list)

Do tập dữ liệu `train.csv` quá lớn sẽ làm cho model training chạy rất chậm, nên ta chỉ dùng 20% dữ liệu của tập `train`

In [4]:
train_types = {"fare_amount": "float32",
              "pickup_datetime": "str", 
              "pickup_longitude": "float32",
              "pickup_latitude": "float32",
              "dropoff_longitude": "float32",
              "dropoff_latitude": "float32",
              "passenger_count": "uint8"}
X_train = load_nyc_taxi_fare(TRAIN_PATH, train_types, 
                              chunksize=5_000_000,
                              datetime_format="%Y-%m-%d %H:%M:%S+00:00")

X_train = X_train.sample(frac=0.2, random_state=210)
X_train.shape

11it [05:07, 27.20s/it]


(10850896, 7)

In [5]:
test_types = train_types.copy()
test_types.pop("fare_amount")
test_types["key"] = "str"
print("test_types:", test_types)

X_test = load_nyc_taxi_fare(TEST_PATH, test_types,
                           datetime_format="%Y-%m-%d %H:%M:%S+00:00")
X_test.shape

test_types: {'pickup_datetime': 'str', 'pickup_longitude': 'float32', 'pickup_latitude': 'float32', 'dropoff_longitude': 'float32', 'dropoff_latitude': 'float32', 'passenger_count': 'uint8', 'key': 'str'}


(9914, 7)

In [6]:
y_train = X_train["fare_amount"]
X_train = X_train.drop(["fare_amount"], axis=1)

In [7]:
y_train.head()

43211180    13.0
27239260     8.1
2232607      5.0
33412474    16.9
31114999    17.0
Name: fare_amount, dtype: float32

In [8]:
X_train.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
43211180,2013-08-15 07:08:42+00:00,-73.985954,40.767220,-73.985107,40.732491,1
27239260,2010-08-25 18:36:00+00:00,-73.940933,40.744080,-73.932442,40.743553,1
2232607,2014-02-18 21:27:29+00:00,-73.992760,40.742912,-74.004753,40.747952,1
33412474,2010-07-07 19:26:00+00:00,-73.979012,40.765575,-74.008499,40.705063,5
31114999,2013-05-07 21:07:00+00:00,-73.978752,40.785679,-73.993744,40.732674,1


In [9]:
X_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982521,40.751259,-73.979652,40.746140,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981163,40.767807,-73.990448,40.751637,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966049,40.789776,-73.988564,40.744427,1


In [10]:
test_key = X_test["key"]
X_test = X_test.drop(["key"], axis=1)

In [11]:
test_key.head()

0    2015-01-27 13:08:24.0000002
1    2015-01-27 13:08:24.0000003
2    2011-10-08 11:53:44.0000002
3    2012-12-01 21:12:12.0000002
4    2012-12-01 21:12:12.0000003
Name: key, dtype: object

In [12]:
X_test.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44+00:00,-73.982521,40.751259,-73.979652,40.746140,1
3,2012-12-01 21:12:12+00:00,-73.981163,40.767807,-73.990448,40.751637,1
4,2012-12-01 21:12:12+00:00,-73.966049,40.789776,-73.988564,40.744427,1


# Feature extraction

Ta collect tất cả các functions tạo ra columns trong `data_cleaning_and_EDA.ipynb` và đặt chúng trong 1 function để dùng cho cả tập `train` và `test`

In [13]:
# return distance in kilometer
def distance(lon1, lat1, lon2, lat2):
    if isinstance(lon1, pd.Series):
        lon1 = lon1.values
    
    if isinstance(lat1, pd.Series):
        lat1 = lat1.values
        
    if isinstance(lon2, pd.Series):
        lon2 = lon2.values
    
    if isinstance(lat2, pd.Series):
        lat2 = lat2.values
        
    # use more precise floating numbers
    if isinstance(lon1, np.ndarray):
        lon1 = np.asarray(lon1, dtype=np.float64)
    
    if isinstance(lat1, np.ndarray):
        lat1 = np.asarray(lat1, dtype=np.float64)
        
    if isinstance(lon2, np.ndarray):
        lon2 = np.asarray(lon2, dtype=np.float64)
    
    if isinstance(lat2, np.ndarray):
        lat2 = np.asarray(lat2, dtype=np.float64)
        
    lon1_rad = np.radians(lon1)
    lat1_rad = np.radians(lat1)
    lon2_rad = np.radians(lon2)
    lat2_rad = np.radians(lat2)
    
    a = 0.5 - 0.5*np.cos(lat2_rad - lat1_rad) + np.cos(lat1_rad)*np.cos(lat2_rad)*(1 - np.cos(lon2_rad - lon1_rad))*0.5
    return 12742 * np.arcsin(np.sqrt(a))

def add_distance_col(df):
    df["distance"] = distance(df["pickup_longitude"], df["pickup_latitude"],
                              df["dropoff_longitude"], df["dropoff_latitude"])
    df["distance"] = df["distance"].astype(np.float32)
    return df

In [14]:
# 40.639722, -73.778889
JFK_LON = -73.778889
JFK_LAT = 40.639722

# 40.6925, -74.168611
EWR_LON = -74.168611
EWR_LAT = 40.6925

# 40.77725, -73.872611
LGA_LON = -73.872611
LGA_LAT = 40.77725

def is_to_airport(df, airport_lon, airport_lat, thres=3):
    dist = distance(airport_lon, airport_lat, df["dropoff_longitude"], df["dropoff_latitude"])
    return dist < thres

def is_from_airport(df, airport_lon, airport_lat, thres=3.):
    dist = distance(airport_lon, airport_lat, df["pickup_longitude"], df["pickup_latitude"])
    return dist < thres

def mark_airport_trip(df, 
                      jfk_lon=JFK_LON, jfk_lat=JFK_LAT, 
                      ewr_lon=EWR_LON, ewr_lat=EWR_LAT,
                      lga_lon=LGA_LON, lga_lat=LGA_LAT):
    
    df["from_to_airport"] = "No"
    
    from_to_jfk = is_to_airport(df, jfk_lon, jfk_lat) | is_from_airport(df, jfk_lon, jfk_lat)
    df.loc[from_to_jfk, "from_to_airport"] = "JFK"
    
    from_to_ewr = is_to_airport(df, ewr_lon, ewr_lat) | is_from_airport(df, ewr_lon, ewr_lat)
    df.loc[from_to_ewr, "from_to_airport"] = "EWR"
    
    from_to_lga = is_to_airport(df, lga_lon, lga_lat) | is_from_airport(df, lga_lon, lga_lat)
    df.loc[from_to_lga, "from_to_airport"] = "LGA"
    
    return df

In [15]:
# New York City Coordinates from google
NYC_LON = -74.006
NYC_LAT = 40.7128

def add_pickup_to_center_dist_col(df, nyc_lon=NYC_LON, nyc_lat=NYC_LAT):
    df["pickup_to_center_dist"] = distance(nyc_lon, nyc_lat, 
                                           df["pickup_longitude"], df["pickup_latitude"])
    df["pickup_to_center_dist"] = df["pickup_to_center_dist"].astype(np.float32)
    return df


def add_dropoff_to_center_dist_col(df, nyc_lon=NYC_LON, nyc_lat=NYC_LAT):
    df["dropoff_to_center_dist"] = distance(nyc_lon, nyc_lat, 
                                            df["dropoff_longitude"], df["dropoff_latitude"])
    df["dropoff_to_center_dist"] = df["dropoff_to_center_dist"].astype(np.float32)
    return df

In [16]:
def direction(lons_1, lats_1, lons_2, lats_2):
    bm = Basemap()
    
    x1, y1 = bm(lons_1, lats_1)
    x2, y2 = bm(lons_2, lats_2)
    
    dx = x2 - x1
    dy = y2 - y1
    
    hypotenuse = np.sqrt(dx*dx + dy*dy)
    
    pos_dx = dx >= 0
    neg_dx = dx < 0
    
    pos_dy = dy >= 0
    neg_dy = dy < 0
    
    neg_dx_and_pos_dy = neg_dx & pos_dy
    neg_dx_and_neg_dy = neg_dx & neg_dy
    
    direc = np.zeros(len(dx))
    
    direc[pos_dx] = np.arcsin(dy[pos_dx] / hypotenuse[pos_dx])
    
    direc[neg_dx_and_pos_dy] = np.pi - np.arcsin(dy[neg_dx_and_pos_dy] / hypotenuse[neg_dx_and_pos_dy])
    
    direc[neg_dx_and_neg_dy] = -np.pi - np.arcsin(dy[neg_dx_and_neg_dy] / hypotenuse[neg_dx_and_neg_dy])
    
    direc = 180 / np.pi * direc
    return direc

def add_direction_col(df):
    df["direction"] = direction(df["pickup_longitude"], df["pickup_latitude"],
                                df["dropoff_longitude"], df["dropoff_latitude"])
    df["direction"] = df["direction"].astype(np.float32)
    
    return df

def impute_nan_direction(df):
    df.loc[df["direction"].isna(), "direction"] = df["direction"].median()
    return df

In [17]:
def ad_time_cols(df):
    df["year"] = df["pickup_datetime"].apply(lambda t: t.year).astype(np.int32)
    df["month"] = df["pickup_datetime"].apply(lambda t: t.month).astype(np.uint8)
    df["weekday"] = df["pickup_datetime"].apply(lambda t: t.weekday).astype(np.uint8)
    df["hour"] = df["pickup_datetime"].apply(lambda t: t.hour).astype(np.uint8)
    return df

In [18]:
COLS_TO_REMOVE = ["pickup_datetime"]

def extract_new_features(df, cols_to_remove=COLS_TO_REMOVE):
    df = add_distance_col(df)
    df = mark_airport_trip(df)
    df = add_pickup_to_center_dist_col(df)
    df = add_dropoff_to_center_dist_col(df)
    df = add_direction_col(df)
    df = impute_nan_direction(df)
    df = ad_time_cols(df)
    
    df = df.drop(cols_to_remove, axis=1)
    return df

In [19]:
X_train = extract_new_features(X_train)
X_test = extract_new_features(X_test)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in arcsin


In [20]:
X_train.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,from_to_airport,pickup_to_center_dist,dropoff_to_center_dist,direction,year,month,weekday,hour
43211180,-73.985954,40.767220,-73.985107,40.732491,1,3.862348,No,6.282433,2.809587,-88.603241,2013,8,3,7
27239260,-73.940933,40.744080,-73.932442,40.743553,1,0.717761,No,6.492984,7.079102,-3.547489,2010,8,2,18
2232607,-73.992760,40.742912,-74.004753,40.747952,1,1.155339,No,3.529324,3.910081,157.209534,2014,2,1,21
33412474,-73.979012,40.765575,-74.008499,40.705063,5,7.172727,No,6.293475,0.885744,-115.979851,2010,7,2,19
31114999,-73.978752,40.785679,-73.993744,40.732674,1,6.027651,No,8.422553,2.439303,-105.792786,2013,5,1,21


In [21]:
X_test.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,from_to_airport,pickup_to_center_dist,dropoff_to_center_dist,direction,year,month,weekday,hour
0,-73.973320,40.763805,-73.981430,40.743835,1,2.323260,No,6.304552,4.024398,-112.102699,2015,1,1,13
1,-73.986862,40.719383,-73.998886,40.739201,1,2.425353,No,1.771281,2.996195,121.246758,2015,1,1,13
2,-73.982521,40.751259,-73.979652,40.746140,1,0.618412,No,4.711865,4.321139,-60.735573,2011,10,5,11
3,-73.981163,40.767807,-73.990448,40.751637,1,1.960778,No,6.464530,4.512865,-119.864113,2012,12,5,21
4,-73.966049,40.789776,-73.988564,40.744427,1,5.387280,No,9.197128,3.811321,-116.402885,2012,12,5,21


One-hot encoding.

In [22]:
CAT_COLS = ("from_to_airport", "month", "weekday", "hour")

def onehot_encode(X_train, X_test, cat_cols=CAT_COLS):
    type_dict = {col: "category" for col in cat_cols}
    
    X_train_ohe = pd.get_dummies(X_train.astype(type_dict), drop_first=True)
    X_test_ohe = pd.get_dummies(X_test.astype(type_dict), drop_first=True)

    X_train_ohe, X_test_ohe = X_train_ohe.align(X_test_ohe, join='inner', axis=1)
    return X_train_ohe, X_test_ohe

In [23]:
X_train, X_test = onehot_encode(X_train, X_test)

In [24]:
X_train.shape

(10850896, 53)

In [25]:
X_train.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,pickup_to_center_dist,dropoff_to_center_dist,direction,year,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
43211180,-73.985954,40.767220,-73.985107,40.732491,1,3.862348,6.282433,2.809587,-88.603241,2013,...,0,0,0,0,0,0,0,0,0,0
27239260,-73.940933,40.744080,-73.932442,40.743553,1,0.717761,6.492984,7.079102,-3.547489,2010,...,0,0,0,0,1,0,0,0,0,0
2232607,-73.992760,40.742912,-74.004753,40.747952,1,1.155339,3.529324,3.910081,157.209534,2014,...,0,0,0,0,0,0,0,1,0,0
33412474,-73.979012,40.765575,-74.008499,40.705063,5,7.172727,6.293475,0.885744,-115.979851,2010,...,0,0,0,0,0,1,0,0,0,0
31114999,-73.978752,40.785679,-73.993744,40.732674,1,6.027651,8.422553,2.439303,-105.792786,2013,...,0,0,0,0,0,0,0,1,0,0


Split train set further into train and evaluation set.

In [26]:
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.2, random_state=210)

Scale the features.

In [27]:
std_scaler = StandardScaler()
std_scaler.fit(X_train)

X_train = std_scaler.transform(X_train)
X_eval = std_scaler.transform(X_eval)
X_test = std_scaler.transform(X_test)

# Linear regression

In [28]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
y_train_pred = lr.predict(X_train)
print("Training RMSE: %0.5f" % np.sqrt(mean_squared_error(y_train, y_train_pred)))

Training RMSE: 4.94890


In [30]:
y_eval_pred = lr.predict(X_eval)
print("Evaluation RMSE: %0.5f" % np.sqrt(mean_squared_error(y_eval, y_eval_pred)))

Evaluation RMSE: 4.95665


In [38]:
pd.DataFrame(test_key.head())

,key
0,2015-01-27 13:08:24.0000002
1,2015-01-27 13:08:24.0000003
2,2011-10-08 11:53:44.0000002
3,2012-12-01 21:12:12.0000002
4,2012-12-01 21:12:12.0000003


In [40]:
def write_submission(test_key, y_test_pred, file_name):
    submit = pd.DataFrame(test_key)
    submit["fare_amount"] = y_test_pred
    submit.to_csv(file_name, index=False)
    return None

In [41]:
y_test_pred = lr.predict(X_test)
write_submission(test_key, y_test_pred, os.path.join(SUB_DIR, "lr_model.csv"))

# Random forest

In [46]:
rf = RandomForestRegressor(n_estimators=10, max_depth=14)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=14,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [47]:
y_train_pred = rf.predict(X_train)
print("Training RMSE: %0.5f" % np.sqrt(mean_squared_error(y_train, y_train_pred)))

Training RMSE: 3.53756


In [48]:
y_eval_pred = rf.predict(X_eval)
print("Evaluation RMSE: %0.5f" % np.sqrt(mean_squared_error(y_eval, y_eval_pred)))

Evaluation RMSE: 3.83345


In [49]:
y_test_pred = rf.predict(X_test)
write_submission(test_key, y_test_pred, os.path.join(SUB_DIR, "rf_model.csv"))